### Exercises
- Data Acquisition
- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
     - Inspect your folder structure. What do you notice?
     
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2. How many Stray Animal cases are there?

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

4. Convert the council_district column to a string column.

5. Extract the year from the case_closed_date column.

6. Convert num_days_late from days to hours in new columns num_hours_late.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?


# Import

In [1]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType

#  Read the case, department, and source data into their own spark dataframes.

# Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
     - Inspect your folder structure. What do you notice?
     

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [4]:
#creating schema
schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)


# Read csv, but now we specify the schema:

source = spark.read.csv("source.csv", header=True, schema=schema)

In [5]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [6]:
#Llooking at values in source
source.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [7]:
# write data to a destination using .write property

source.write.json("source_json", mode="overwrite")

# Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


In [8]:
# Looking at data types
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [9]:
# Read the case.csv file

df = spark.read.csv("case.csv", header=True, inferSchema=True)

In [10]:
# look at first three records
df.show(3, truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [11]:
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [12]:
#changing case_id to a string
df = df.withColumn('case_id', col('case_id').cast('string'))

In [13]:
#changing district_id to a string
df = df.withColumn('council_district', col('council_district').cast('string'))

# 1. How old is the latest  currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [38]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [26]:
df[['case_id','case_opened_date', 'case_closed_date', 'case_status', 'SLA_days']].where(col('case_status') != 'Closed').sort(desc('SLA_days')).show(1)

+----------+----------------+----------------+-----------+----------+
|   case_id|case_opened_date|case_closed_date|case_status|  SLA_days|
+----------+----------------+----------------+-----------+----------+
|1013896575|    9/22/17 8:27|            null|       Open|1419.00191|
+----------+----------------+----------------+-----------+----------+
only showing top 1 row



In [24]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?
df[['case_id','case_opened_date', 'case_closed_date', 'case_status', 'SLA_days']].where(col('case_status') != 'Closed').sort(asc('SLA_days')).show(1)

+----------+----------------+----------------+-----------+--------+
|   case_id|case_opened_date|case_closed_date|case_status|SLA_days|
+----------+----------------+----------------+-----------+--------+
|1013810864|   8/21/17 15:22|            null|       Open|    null|
+----------+----------------+----------------+-----------+--------+
only showing top 1 row



In [53]:
# convert the number of days a case is late to a number of weeks

df = df.withColumn('num_weeks_late', expr('num_days_late/7'))

df.select("num_days_late", "num_weeks_late").show()

+-------------------+--------------------+
|      num_days_late|      num_weeks_late|
+-------------------+--------------------+
| -998.5087616000001|        -142.6441088|
|-2.0126041669999997|-0.28751488099999994|
|       -3.022337963|-0.43176256614285713|
|       -15.01148148| -2.1444973542857144|
|0.37216435200000003|         0.053166336|
|       -29.74398148|  -4.249140211428571|
|       -14.70673611| -2.1009623014285714|
|       -14.70662037|  -2.100945767142857|
|       -14.70662037|  -2.100945767142857|
|       -14.70649306|         -2.10092758|
|       -14.70649306|         -2.10092758|
|       -14.70636574| -2.1009093914285715|
|          -14.70625|  -2.100892857142857|
|       -14.70636574| -2.1009093914285715|
|       -14.70623843| -2.1008912042857144|
|-14.705891199999998|          -2.1008416|
|       -14.70600694|  -2.100858134285714|
|       -14.70576389|  -2.100823412857143|
|       -14.70576389|  -2.100823412857143|
|       -14.70564815| -2.1008068785714284|
+----------

In [58]:
df.select('case_opened_date', 'case_closed_date', 'case_due_date', 'num_days_late').show()

+----------------+----------------+-------------+-------------------+
|case_opened_date|case_closed_date|case_due_date|      num_days_late|
+----------------+----------------+-------------+-------------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42| -998.5087616000001|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|-2.0126041669999997|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|       -3.022337963|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|       -15.01148148|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|0.37216435200000003|
|     1/1/18 6:28|    1/1/18 14:38| 1/31/18 8:30|       -29.74398148|
|     1/1/18 6:57|    1/2/18 15:32| 1/17/18 8:30|       -14.70673611|
|     1/1/18 6:58|    1/2/18 15:32| 1/17/18 8:30|       -14.70662037|
|     1/1/18 6:58|    1/2/18 15:32| 1/17/18 8:30|       -14.70662037|
|     1/1/18 6:59|    1/2/18 15:32| 1/17/18 8:30|       -14.70649306|
|     1/1/18 7:00|    1/2/18 15:32| 1/17/18 8:30|       -14.70649306|
|     1/1/18 7:02|  

# 2. How many Stray Animal cases are there?

In [54]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'num_weeks_late']

In [62]:
# Looking at certain columns
df.select('case_opened_date', 'case_closed_date', 'case_due_date',  'service_request_type').show()

+----------------+----------------+-------------+--------------------+
|case_opened_date|case_closed_date|case_due_date|service_request_type|
+----------------+----------------+-------------+--------------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|        Stray Animal|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|Removal Of Obstru...|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|Removal Of Obstru...|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|Front Or Side Yar...|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|Animal Cruelty(Cr...|
|     1/1/18 6:28|    1/1/18 14:38| 1/31/18 8:30|Traffic Signal Op...|
|     1/1/18 6:57|    1/2/18 15:32| 1/17/18 8:30|Front Or Side Yar...|
|     1/1/18 6:58|    1/2/18 15:32| 1/17/18 8:30|Front Or Side Yar...|
|     1/1/18 6:58|    1/2/18 15:32| 1/17/18 8:30|Right Of Way/Side...|
|     1/1/18 6:59|    1/2/18 15:32| 1/17/18 8:30|Front Or Side Yar...|
|     1/1/18 7:00|    1/2/18 15:32| 1/17/18 8:30|Front Or Side Yar...|
|     

In [91]:
df.where(df["service_request_type"] == "Stray Animal").groupby(df.service_request_type)\
.agg(count(lit('service_request_type'))).show()

+--------------------+---------------------------+
|service_request_type|count(service_request_type)|
+--------------------+---------------------------+
|        Stray Animal|                      26760|
+--------------------+---------------------------+



# 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [27]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [50]:
df[['dept_division','service_request_type']].count()

841704

In [49]:
df.filter(col('dept_division') == lit('Field Operations')).filter(col('service_request_type') != lit('Officer Standby')).count()

113902

# 4. Convert the council_district column to a string column.

In [51]:
# council_district as a string instead of int
df = df.withColumn('council_district', col('council_district').cast('string'))

In [54]:
df[['council_district']].dtypes

[('council_district', 'string')]

# 5. Extract the year from the case_closed_date column.


In [68]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [69]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

df = df.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('SLA_due_date', to_timestamp('SLA_due_date', fmt))

In [71]:
#Looking at the case closed column
df[['case_closed_date']].show()

+-------------------+
|   case_closed_date|
+-------------------+
|2018-01-01 12:29:00|
|2018-01-03 08:11:00|
|2018-01-02 07:57:00|
|2018-01-02 08:13:00|
|2018-01-01 13:29:00|
|2018-01-01 14:38:00|
|2018-01-02 15:32:00|
|2018-01-02 15:32:00|
|2018-01-02 15:32:00|
|2018-01-02 15:32:00|
|2018-01-02 15:32:00|
|2018-01-02 15:32:00|
|2018-01-02 15:33:00|
|2018-01-02 15:32:00|
|2018-01-02 15:33:00|
|2018-01-02 15:33:00|
|2018-01-02 15:33:00|
|2018-01-02 15:33:00|
|2018-01-02 15:33:00|
|2018-01-02 15:33:00|
+-------------------+
only showing top 20 rows



In [117]:
df.withColumn('year', year('case_closed_date')).show(vertical= True)

-RECORD 0--------------------------------------
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 SLA_due_date           | 2020-09-26 00:42:00  
 case_late              | NO                   
 num_days_late          | -41.60453173333334   
 case_closed            | YES                  
 dept_division          | Field Operations     
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 source_id              | svcCRMLS             
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
 year                   | 2018                 
-RECORD 1-------------------------------

# 6. Convert num_days_late from days to hours in new columns num_hours_late.


In [75]:
# Convert num_days_late from days to hours in new columns num_hours_late.

df = df.withColumn('num_days_late', expr('num_days_late/24'))

df.select("num_days_late").show(5)

+--------------------+
|       num_days_late|
+--------------------+
|  -41.60453173333334|
|-0.08385850695833331|
|-0.12593074845833332|
|        -0.625478395|
|0.015506848000000002|
+--------------------+
only showing top 5 rows



# 7. Join the case data with the source and department data.


In [77]:
dept= spark.read.csv("dept.csv", header=True, inferSchema=True)

In [80]:
df = df.join(dept, on=dept.dept_division == df.dept_division).drop(dept.dept_division)
df = df.join(source, on = df.source_id== source.source_id).drop(source.source_id)

# 8. Are there any cases that do not have a request source?

There is none


In [119]:
df.filter(col('source_id') == lit('Null')).show(vertical= True)

(0 rows)



# 9. What are the top 10 service request types in terms of number of requests?



In [84]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'dept_name',
 'standardized_dept_name',
 'dept_subject_to_SLA',
 'source_username']

In [104]:
df[['service_request_type']].groupby('service_request_type').count().sort(desc('count')).show(10, truncate= False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



# 10. What are the top 10 service request types in terms of average days late?


In [110]:
df.groupby('service_request_type').agg(mean('num_days_late').alias('avg days late')).sort(desc('avg days late')).show(10, truncate= False)

+--------------------------------------+------------------+
|service_request_type                  |avg days late     |
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |7.331515087675396 |
|Labeling for Used Mattress            |6.767930375952381 |
|Record Keeping of Used Mattresses     |6.416551683095237 |
|Signage Requied for Sale of Used Mattr|6.3182783563888885|
|Storage of Used Mattress              |5.921356517291668 |
|Zoning: Recycle Yard                  |5.663688171866582 |
|Donation Container Enforcement        |5.489837710982796 |
|License Requied Used Mattress Sales   |5.366595293392857 |
|Traffic Signal Graffiti               |4.241602525916667 |
|Complaint                             |3.036270929296539 |
+--------------------------------------+------------------+
only showing top 10 rows



# 11. Does number of days late depend on department?



In [112]:
df.groupby('dept_division').agg(mean(col('num_days_late'))).sort(desc('avg(num_days_late)')).show(20)

+--------------------+--------------------+
|       dept_division|  avg(num_days_late)|
+--------------------+--------------------+
|Code Enforcement ...|   5.663688171866582|
|        Reservations|       2.75129846625|
|     311 Call Center|  2.4890454790125323|
|Director's Office...|  1.5654436126229208|
|Engineering Division|  0.5597385231612378|
|               Shops|  0.4017192403634455|
|           Tree Crew| 0.19680345050272494|
|         Solid Waste| 0.14662599666150938|
|              Trades|  0.1346657142198722|
|Clean and Green N...| 0.07047787164532521|
|              Vector|-0.04669388747176437|
|    Facility License|-0.05886224042590...|
|       Miscellaneous|-0.07174407016219451|
|    Waste Collection|-0.09044384326999555|
|     Clean and Green| -0.1065481241355893|
|               Brush|-0.16607460714404573|
| Food Establishments|-0.29048134876881365|
|  Signs and Markings| -0.3103595000565716|
|    Shops (Internal)|-0.34109446332818555|
|         Storm Water| -0.585653

# 12. How do number of days late depend on department and request type?

In [113]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'dept_name',
 'standardized_dept_name',
 'dept_subject_to_SLA',
 'source_username']

In [114]:
df.groupby('dept_division', 'service_request_type').agg(mean(col('num_days_late'))).sort(desc('avg(num_days_late)')).show(20)

+--------------------+--------------------+------------------+
|       dept_division|service_request_type|avg(num_days_late)|
+--------------------+--------------------+------------------+
|    Code Enforcement|  Zoning: Junk Yards| 7.331515087675396|
|Code Enforcement ...|Labeling for Used...| 6.767930375952381|
|Code Enforcement ...|Record Keeping of...| 6.416551683095237|
|Code Enforcement ...|Signage Requied f...|6.3182783563888885|
|Code Enforcement ...|Storage of Used M...| 5.921356517291668|
|Code Enforcement ...|Zoning: Recycle Yard| 5.663688171866582|
|    Code Enforcement|Donation Containe...| 5.489837710982796|
|Code Enforcement ...|License Requied U...| 5.366595293392857|
|             Signals|Traffic Signal Gr...| 4.241602525916667|
|Code Enforcement ...|Used Mattress Ins...| 3.872320857014705|
|Code Enforcement ...|        Bandit Signs|3.6507350501333335|
|     311 Call Center|           Complaint| 3.036270929296539|
|    Code Enforcement|             Vendors| 2.772837457